In [1]:
import json
import math
import pandas as pd
import numpy as np
import os, sys
import datetime
from datetime import date
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
#from ggplot import *

from tableausdk import *
from tableausdk.HyperExtract import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#%matplotlib inline

# Initial Re-factoring of the variables

First of all study the different types of variables and make the necessary transformations to create columns that have correct data to study and compare.

In [2]:
def initial_setup():
    """
    Create Initial setup of directories variables, and dataframe vars to use.
    Returns:
      A tuple containing:
          - datadir:   Absolute Path to the data directory of the project.
          - dirname:   Absolute Path of directory that contains this file.
          - imagesdir: Absolute path of directory that contains the images.
          - colnames: A list containing the initial colnames of the dataframe.
    """
    # Initial directories set up
    dirname = os.path.dirname(os.path.abspath('__file__'))
    datadir =  os.path.join(os.path.abspath(os.path.join(os.path.join(dirname, os.pardir), os.pardir)), 'data')
    imagesdir =  os.path.join(os.path.abspath(os.path.join(dirname, os.pardir)), 'images')
    initial_colnames = sorted(['backers_count', 'blurb', 'category', 'country', 'created_at', 'currency', 'deadline', 'goal', 'id', 'launched_at', 'location', 'pledged', 'slug', 'spotlight', 'staff_pick', 'state', 'static_usd_rate', 'usd_pledged'])
    return dirname, datadir, imagesdir, initial_colnames

# 0 - Initial directories and colnames set up
print("Step 0: Initial directories and colnames set up")
dirname, datadir, imagesdir, initial_colnames = initial_setup()
print("Directory of this file is {}".format(dirname))
print("Data directory is {}".format(datadir))
print("Images directory is {}".format(imagesdir))
print("Initial columns for our model are: \n{}".format(initial_colnames))

Step 0: Initial directories and colnames set up
Directory of this file is /home/agericke/crowdfunding_ml/src/formatting
Data directory is /home/agericke/crowdfunding_ml/data
Images directory is /home/agericke/crowdfunding_ml/src/images
Initial columns for our model are: 
['backers_count', 'blurb', 'category', 'country', 'created_at', 'currency', 'deadline', 'goal', 'id', 'launched_at', 'location', 'pledged', 'slug', 'spotlight', 'staff_pick', 'state', 'static_usd_rate', 'usd_pledged']


In [3]:
def read_from_disk(filename):
    """
    Read a dataframe from a filename in disk.
    Params:
        filename....Path to the file.
    Returns:
        A pandas dataframe.
    """
    return pickle.load(open(filename, 'rb'))


def store_dataframe(dataframe, filename):
    """
    Store the dataframe using pickle.
    Params:
        dataframe...pandas dataframe to store.
        filename....Path to the file to store the datafram in.
    Returns:
        Nothing.
    """
    pickle.dump(dataframe, open(filename, 'wb'))
    
# 1 - Load from disk the complete Merged Dataframe.
print("\n\n\nStep 1: Load from disk the complete Merged Dataframe.")
filename = os.path.join(datadir, 'dataframe_total.pkl')
print("Completed Dataframe read from file {}".format(filename))
data = read_from_disk(filename)
# Print summary of dataframe
print("Dataframe contains {} projects and {} columns for each project\n".format(data.shape[0], data.shape[1]))




Step 1: Load from disk the complete Merged Dataframe.
Completed Dataframe read from file /home/agericke/crowdfunding_ml/data/dataframe_total.pkl
Dataframe contains 344209 projects and 18 columns for each project



In [4]:
def remove_cols(dataframe, cols_to_remove):
    """
    Remove all the columns specified by the list from dataframe
    Params:
        cols_to_remove....List of columns we want to remove
        dataframe.........The dataframe to remove the columns from.
    Returns:
        A dataframe with only the columns we want.
    """
    dataframe.drop(cols_to_remove, inplace=True, axis=1)
    print("Succesfully removed columns {}".format(cols_to_remove))
    return dataframe

def categorical_with_per_count(dataframe, feature):
    """
    Calculate frequency of the categorical feature with % and count base.
    Sorted on the descending order.

    Params:
        dataframe.....Pandas dataframe from where to pick the data.
        feature.......Column for which we want to calculate the data for.
    """
    
    # calculate frequency on % and value
    freq_merged = pd.concat([dataframe[feature].value_counts(normalize=True) * 100,
                             dataframe[feature].value_counts(normalize=False)], axis=1)
    # rename columns
    freq_merged.columns = [feature + '_%', feature + '_count']
    return freq_merged

In [5]:
data.head(3)

,backers_count,blurb,category,country,created_at,currency,deadline,goal,id,launched_at,location,pledged,slug,spotlight,staff_pick,state,static_usd_rate,usd_pledged
2967,1,"To create art, an artist must have their tools...","{""urls"":{""web"":{""discover"":""http://www.kicksta...",US,1428557824,USD,1429770591,400.0,2141932586,1428560991,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",20.0,tools-of-the-trade,False,False,canceled,1.0,20.0
4106,0,"Firestarter kits for revolutionaries! markers,...","{""urls"":{""web"":{""discover"":""http://www.kicksta...",US,1453437274,USD,1455244263,63000.0,569937661,1453948263,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",0.0,project-flint,False,False,live,1.0,0.0
635,3,Sophie is an aspiring artist! She will be 5 in...,"{""urls"":{""web"":{""discover"":""http://www.kicksta...",US,1409805071,USD,1412906207,1000.0,1619161443,1410314207,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",120.0,4-year-old-creates-art-for-her-future-education,False,False,failed,1.0,120.0


In [6]:
def check_missing_values_and_drop(data, drop=False):
    """
    Check the number of missing values that we have. Notice that this function
    will count 
    Params:
        data....Dataframe to check the missing values.
        drop....Boolean to indicate if we want to drop missing values or not.
    Returns:
        Prints a summary of the number and % of missing values.
        The dataframe with no missing values
    """
    #dataMis=data[data.isnull().any(axis=1)]
    #print(dataMis.groupby('country').country.count())
    total_rows = data.shape[0]
    #isna() sets True NA values and numpy.NaN. Empty strings or infinites are not set as True.
    na_col_counts = data.isna().sum().sort_values(ascending = False)
    freq_merged = pd.concat([na_col_counts, (na_col_counts/total_rows)*100], axis=1)
    freq_merged.columns = ['Total_count', '%_count']
    print(freq_merged)

    if drop:
        data = data.dropna()
    
    return data
    # TODO: See if we can check the missing indexes for each column and run a study on them.
    # TODO: Run experiments to try to identify is the missing values are mainly because of a reason or one type of project, or specific to one period of time (see if they are missing at random, missing not at random...)

# 2 - Look for missing values for every row and print summary.
print("\n\n\nStep 2: Look for missing values for every row and print summary.")
data = check_missing_values_and_drop(data, False)
print("As we can see, we have very low percentage of missing values,the highest column with missing \
values is location column with only a 0.31%, so we decided to drop the missing values.")
print("Also, studying the missing data, we discover that out of 1091 rows with missing data: \n")
print("United States     1087\nGreat Britain     2\nDenmark           1\nAustria           1\n")
print("The distribution of the missing values across the main_category variable is:\n")
print("art             118\ncomics           14\ncrafts            9\ndance            18\ndesign           12\nfashion           6\nfilm & video    279\nfood            10\ngames            49\njournalism       55\nmusic           258\nphotography      49\npublishing      141\ntechnology       51\ntheater          22")




Step 2: Look for missing values for every row and print summary.
                 Total_count   %_count
location                1072  0.311439
blurb                     18  0.005229
usd_pledged                0  0.000000
goal                       0  0.000000
category                   0  0.000000
country                    0  0.000000
created_at                 0  0.000000
currency                   0  0.000000
deadline                   0  0.000000
id                         0  0.000000
static_usd_rate            0  0.000000
launched_at                0  0.000000
pledged                    0  0.000000
slug                       0  0.000000
spotlight                  0  0.000000
staff_pick                 0  0.000000
state                      0  0.000000
backers_count              0  0.000000
As we can see, we have very low percentage of missing values,the highest column with missing values is location column with only a 0.31%, so we decided to drop the missing values.
Also, study

In [7]:
data.isna().sum()

backers_count         0
blurb                18
category              0
country               0
created_at            0
currency              0
deadline              0
goal                  0
id                    0
launched_at           0
location           1072
pledged               0
slug                  0
spotlight             0
staff_pick            0
state                 0
static_usd_rate       0
usd_pledged           0
dtype: int64

In [8]:
df_null = data[data.isna().any(axis=1)]
data = check_missing_values_and_drop(data, True)

                 Total_count   %_count
location                1072  0.311439
blurb                     18  0.005229
usd_pledged                0  0.000000
goal                       0  0.000000
category                   0  0.000000
country                    0  0.000000
created_at                 0  0.000000
currency                   0  0.000000
deadline                   0  0.000000
id                         0  0.000000
static_usd_rate            0  0.000000
launched_at                0  0.000000
pledged                    0  0.000000
slug                       0  0.000000
spotlight                  0  0.000000
staff_pick                 0  0.000000
state                      0  0.000000
backers_count              0  0.000000


In [9]:
print(data.info())
print(data.get_dtype_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343119 entries, 2967 to 3281
Data columns (total 18 columns):
backers_count      343119 non-null int64
blurb              343119 non-null object
category           343119 non-null object
country            343119 non-null object
created_at         343119 non-null int64
currency           343119 non-null object
deadline           343119 non-null int64
goal               343119 non-null float64
id                 343119 non-null int64
launched_at        343119 non-null int64
location           343119 non-null object
pledged            343119 non-null float64
slug               343119 non-null object
spotlight          343119 non-null bool
staff_pick         343119 non-null bool
state              343119 non-null object
static_usd_rate    343119 non-null float64
usd_pledged        343119 non-null float64
dtypes: bool(2), float64(4), int64(5), object(7)
memory usage: 45.2+ MB
None
bool       2
float64    4
int64      5
object     7
dtype: i

In [10]:
# Change state column to result
data['result'] = data['state']
data.drop('state', inplace=True, axis=1)

In [11]:
# Create goal_usd
data['goal_usd'] = data['goal']*data['static_usd_rate']
data.goal_usd.head(10)

2967       400.00000
4106     63000.00000
635       1000.00000
138       7293.70130
1456      7137.31045
1457    176409.77750
1458      5000.00000
1459     40000.00000
1475      1500.00000
1476    150000.00000
Name: goal_usd, dtype: float64

In [12]:
# Create pledged_usd
data['pledged_usd'] = data['pledged']*data['static_usd_rate']
data.pledged_usd.head(10)

2967        20.000000
4106         0.000000
635        120.000000
138     537049.814122
1456         0.000000
1457         0.000000
1458         0.000000
1459      2180.000000
1475         0.000000
1476        31.000000
Name: pledged_usd, dtype: float64

In [13]:
# Study differences from calculated to the value we had.
print(abs(data['usd_pledged'] - data['pledged_usd']).describe())

# Remove not any more useful cols.
data = remove_cols(data, ['goal', 'pledged', 'static_usd_rate', 'usd_pledged'])

count    3.431190e+05
mean     1.684488e-13
std      3.711379e-12
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      9.313226e-10
dtype: float64
Succesfully removed columns ['goal', 'pledged', 'static_usd_rate', 'usd_pledged']


In [14]:
# Create date type variables for created_at, launched_at and deadline
data['launched_at'] = pd.to_datetime(data['launched_at'], unit='s')
data['deadline'] = pd.to_datetime(data['deadline'], unit='s')
data['created_at'] = pd.to_datetime(data['created_at'], unit='s')

In [22]:
data.get_dtype_counts()
data.columns

Index(['backers_count', 'blurb', 'category', 'country', 'created_at',
       'currency', 'deadline', 'id', 'launched_at', 'location', 'slug',
       'spotlight', 'staff_pick', 'result', 'goal_usd', 'pledged_usd'],
      dtype='object')

In [23]:
from tableausdk.Exceptions import *

# You cannot delete or append to a .tde file if it is opened in Tableau
def to_tde(dataframe, filename, tb_name='Kickstarter'):
    """
    Function for creating an hyper dataset for Tableua from the pandas dataframe.
    
    Params:
        - dataframe.....Pandas dataframe to create the extract from.
        - filename......Path to the file where we will store the .hyper extract.
        - tb_name.......Name of the table to be created. (Deafault: Kickstarter)
        
    Returns:
        Nothing
    """    
    # 0 - Initialize extract API
    ExtractAPI.initialize()

    # Step 1: Create the Extract File
    dataExtract = Extract(filename)

    if dataExtract.hasTable(tb_name):
        return print("tde already exist use another name")

    # Step 2: Create the table definition
    if (not dataExtract.hasTable(tb_name)):
        dataSchema = TableDefinition()
        dataSchema.addColumn('backers_count', Type.INTEGER)
        dataSchema.addColumn('blurb', Type.UNICODE_STRING)
        dataSchema.addColumn('category', Type.CHAR_STRING)
        dataSchema.addColumn('country', Type.CHAR_STRING)
        dataSchema.addColumn('created_at', Type.DATETIME)
        dataSchema.addColumn('currency', Type.CHAR_STRING)
        dataSchema.addColumn('deadline', Type.DATETIME)
        dataSchema.addColumn('id', Type.INTEGER)
        dataSchema.addColumn('launched_at', Type.DATETIME)
        dataSchema.addColumn('location', Type.UNICODE_STRING)
        dataSchema.addColumn('slug', Type.UNICODE_STRING)
        dataSchema.addColumn('spotlight', Type.BOOLEAN)
        dataSchema.addColumn('staff_pick', Type.BOOLEAN)
        dataSchema.addColumn('state', Type.CHAR_STRING)
        dataSchema.addColumn('goal_usd', Type.DOUBLE)
        dataSchema.addColumn('pledged_usd', Type.DOUBLE)

        # Step 3: Create a table in the image of the table definition
        table = dataExtract.addTable(tb_name, dataSchema)

    # Step 4: Create rows and insert them one by one
    newRow = Row(dataSchema)
    for i in range(0, len(dataframe)):
        newRow.setInteger(0, dataframe['backers_count'].iloc[i])
        newRow.setString(1, dataframe['blurb'].iloc[i])
        newRow.setCharString(2, dataframe['category'].iloc[i])
        newRow.setCharString(3, dataframe['country'].iloc[i])
        newRow.setDateTime(4, dataframe['created_at'].iloc[i].year, dataframe['created_at'].iloc[i].month, dataframe['created_at'].iloc[i].day, dataframe['created_at'].iloc[i].hour, dataframe['created_at'].iloc[i].minute, dataframe['created_at'].iloc[i].second, dataframe['created_at'].iloc[i].microsecond)
        newRow.setCharString(5, dataframe['currency'].iloc[i])
        newRow.setDateTime(6, dataframe['deadline'].iloc[i].year, dataframe['deadline'].iloc[i].month, dataframe['deadline'].iloc[i].day, dataframe['deadline'].iloc[i].hour, dataframe['deadline'].iloc[i].minute, dataframe['deadline'].iloc[i].second, dataframe['deadline'].iloc[i].microsecond)
        newRow.setInteger(7, dataframe['id'].iloc[i])
        newRow.setDateTime(8, dataframe['launched_at'].iloc[i].year, dataframe['launched_at'].iloc[i].month, dataframe['launched_at'].iloc[i].day, dataframe['launched_at'].iloc[i].hour, dataframe['launched_at'].iloc[i].minute, dataframe['launched_at'].iloc[i].second, dataframe['launched_at'].iloc[i].microsecond)
        newRow.setString(9, dataframe['location'].iloc[i])
        newRow.setString(10, dataframe['slug'].iloc[i])
        newRow.setBoolean(11, dataframe['spotlight'].iloc[i])
        newRow.setBoolean(12, dataframe['staff_pick'].iloc[i])
        newRow.setCharString(13, dataframe['state'].iloc[i])
        newRow.setDouble(14, dataframe['goal_usd'].iloc[i])
        newRow.setDouble(15, dataframe['pledged_usd'].iloc[i])
        
        table.insert(newRow)

    # Step 5: Close the tde
    dataExtract.close()
    
    # Step 6: Close the Tableau Extract API
    ExtractAPI.cleanup()

In [24]:
# to_tde(data, os.path.join(datadir, 'kickstarter.hyper'))

In [25]:
def obtain_cat_and_subcat_for_row(row):
    """
    Obtain category and subcategory of a row. notice that if subcategory not present,
    this will set it to none.
    Params:
        row....The dataframe row.
    Returns:
        A tuple containing:
            - category: Value for obtained category.
            - subcategory: Value for obtained subcategory, and None if not present.
    """
    # Convert string to dict
    cat_split = json.loads(row['category'])['slug'].split('/')
    # Save category and subcategory
    category = cat_split[0]
    subcategory = ""
    
    if len(cat_split)>1:
        # There is a sucategory
        subcategory = cat_split[1]
    
    return (category, subcategory)
    

def create_cat_and_subcat(data):
    """
    Save in a new column the category and subcategory extracted from the category column.
    Params:
        data.....Dataframe.
    Returns:
        A dataframe with category and subcategory vars created.¡ for each row.
    """
    cat_subcat = data.apply(obtain_cat_and_subcat_for_row, axis=1)
    data['main_category'] = [c[0].lower().strip().replace(' ','-') for c in cat_subcat]
    data['sub_category'] = [c[1].lower().strip().replace(' ','-') for c in cat_subcat]
    data.drop('category', inplace=True, axis=1)
    print('Succesfully created columns category and subcategory')
    
    return data

data = create_cat_and_subcat(data)
data.head(2)

Succesfully created columns category and subcategory


,backers_count,blurb,country,created_at,currency,deadline,id,launched_at,location,slug,spotlight,staff_pick,result,goal_usd,pledged_usd,main_category,sub_category
2967,1,"To create art, an artist must have their tools...",US,2015-04-09 05:37:04,USD,2015-04-23 06:29:51,2141932586,2015-04-09 06:29:51,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",tools-of-the-trade,False,False,canceled,400.0,20.0,art,digital-art
4106,0,"Firestarter kits for revolutionaries! markers,...",US,2016-01-22 04:34:34,USD,2016-02-12 02:31:03,569937661,2016-01-28 02:31:03,"{""country"":""US"",""urls"":{""web"":{""discover"":""htt...",project-flint,False,False,live,63000.0,0.0,crafts,printing


In [26]:
def obtain_location_vars_for_row(row):
    """
    Obtain country, state and city.
    Params:
        row....The dataframe row.
    Returns:
        A tuple containing:
            - country: Country oobtained.
            - state: State obtained.
            - loc_type: Type of location obtained.
    """
    # Convert string to dict
    loc_dict = json.loads(row['location'])
    country = loc_dict['country']
    state = loc_dict['state']
    city =  loc_dict['name']
    
    return (country, state, city)
    

def create_location_vars(data):
    """
    Save in a new column the country, state and location type extracted from the location column.
    Params:
        data.....Dataframe.
    Returns:
        A dataframe with country, state and location vars created for each row.
    """
    location_vars = data.apply(obtain_location_vars_for_row, axis=1)
    data['country2'] = [c[0] for c in location_vars]
    data['state'] = [c[1] for c in location_vars]
    data['city'] = [c[2] for c in location_vars]
    #data.drop("location", inplace=True, axis=1)
    print("Succesfully created columns country, region_state and type")
    
    return data

In [27]:
df2 = create_location_vars(data)
data[data.country != data.country2]
# After some comparison we determine that country2 is more realiable than country.
data['country'] = data['country2']
data.drop('country2', inplace=True, axis=1)

Succesfully created columns country, region_state and type


# EDA (Exploratory Data Analysis)

After we have set all columns correctly we proceed to perform a EDA study of all vars. This study will cover:

- Study of general data types, unique identifier and missing values.
- Perform per var analysis of ranges, check categorical variables.
- Per variable target variable analysis.
- Possible variable relations.
- Variable cleaning.
- Dataset set-up for modelling

In [28]:
# 1 - Overview of the data.
print(data.info())
print('\n')
print(data.get_dtype_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343119 entries, 2967 to 3281
Data columns (total 19 columns):
backers_count    343119 non-null int64
blurb            343119 non-null object
country          343119 non-null object
created_at       343119 non-null datetime64[ns]
currency         343119 non-null object
deadline         343119 non-null datetime64[ns]
id               343119 non-null int64
launched_at      343119 non-null datetime64[ns]
location         343119 non-null object
slug             343119 non-null object
spotlight        343119 non-null bool
staff_pick       343119 non-null bool
result           343119 non-null object
goal_usd         343119 non-null float64
pledged_usd      343119 non-null float64
main_category    343119 non-null object
sub_category     343119 non-null object
state            343023 non-null object
city             343119 non-null object
dtypes: bool(2), datetime64[ns](3), float64(2), int64(2), object(10)
memory usage: 47.8+ MB
None


int64    

In [30]:
# 2 - Check key for the data
# Check that id columns can be used as a key and that has only one count per value
(data.id.value_counts() > 1).sum()
# After we check that can be used as key we set it as index.
data.set_index('id', inplace=True)

In [ ]:
# 2 - Check new missing values
data.isna().sum().sort_values(ascending=False)

In [ ]:
# Check if blurb lenght has any power over project success.

# Study backers_count variable. (Scatterplot and check medium value of backer.)

# Study per country rates of success.

# Study per year, month, time of the year and day of the week rate of success for creation date.

# Study per currency rate of success.

# Study per year, month, time of the year and day of the week rate of success for creation date.